In [1]:
# dataset prep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

data = pd.read_csv('data.csv')
data=data.drop(data.index[13])
Y1 = data.iloc[:,0:1]
X1 = data.iloc[:,1:8267].values
Y1.columns=['Label']

Y = pd.DataFrame(((Y1["Label"].astype(str)).str.split("_", 1)).str.get(0), columns = ['Label']).values

idf = np.zeros([1,X1.shape[1]])
df = np.zeros([1,X1.shape[1]])

for i in range(8266):
    for j in range(589):
        if X1[j,i]!=0:
            df[0,i]=df[0,i]+1

for i in range(8266):
    idf[0,i]=(np.log(590/(1+df[0,i])))

X2=np.zeros([X1.shape[0],X1.shape[1]])
for i in range(589):
    for j in range(8266):
        X2[i,j]=X1[i,j]*idf[0,j]

sq=np.square(X2)

sumsq=np.zeros([X1.shape[0],1])
for i in range(589):
    sumsq[i,0]=np.sqrt(np.sum(sq[i]))
    

X=np.zeros([X1.shape[0],X1.shape[1]])
for i in range(589):
    for j in range(8266):
        X[i,j]=X2[i,j]/sumsq[i,0]


In [2]:
#turning the string labels into numerical labels
count=0
label = np.zeros([589,1])
label[0,0]=0
for i in range(1,588):
    if Y[i]!=Y[i-1]:
        count=count+1
    label[i]=count

In [3]:
#counting the frequency of each label
freq=np.zeros([1,8])

for i in range(589):
    freq[0,(int)(label[i][0])]=freq[0,(int)(label[i][0])]+1

labelfreq=(freq[0]).tolist()

In [4]:
def tlog(x):
    if x:
        return x*(np.log2(x))
    else :
        return 0

In [5]:
#function to find the entropy of a list of frequencies 
def entlist(lis,n):
    sum=np.sum(lis)
    ent=0
    for i in range(n):
        ent=ent-tlog(lis[i]/sum)
    return ent   
        

In [6]:
#function to find the number in each cluster
def freqclus(lis):
    clusfreq=[]

    for i in range(8):
        clusfreq.append(len(lis[i]))
    return clusfreq

In [7]:
#function to find the freq of each label in a cluster 
def classbyclust(lis):
    clustlabels=np.zeros([1,8])
    for i in lis:
        clustlabels[0,(int)(label[i])]=clustlabels[0,(int)(label[i])]+1
    return (clustlabels[0]).tolist()
        
    
    

In [8]:
#function to find the weighted sum of entropies of clusters 
def condent(lis):
    sum1=0
    for i in range(8):
        f=classbyclust(lis[i])
        sum1=sum1+(entlist(f,8))*(len(lis[i]))/(589)
    return sum1
    

In [9]:
#nmi_calculation
def nmicalc(lis):
    nmi=2*(entlist(labelfreq,8)-condent(lis))/(entlist(freqclus(lis),8)+entlist(labelfreq,8))
    return nmi

In [10]:
%store -r pca_agglo_final_clust

In [11]:
%store -r agglo_final_clust

In [12]:
%store -r kmeans_final_clust

In [13]:
%store -r pca_kmeans_final_clust

In [14]:
nmicalc(agglo_final_clust)

0.024552195075452997

In [15]:
nmicalc(pca_agglo_final_clust)

0.023201837753411

In [16]:
nmicalc(kmeans_final_clust)

0.5063931099623016

In [17]:
nmicalc(pca_kmeans_final_clust)

0.6858655473657979